In [ ]:
#@title 🎧 Code Walkthrough: Setup Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_01_setup_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

In [ ]:
#@title 🎧 Download Narration Audio & Play Introduction
import os as _os
if not _os.path.exists("/content/narration"):
    !pip install -q gdown
    import gdown
    gdown.download(id="1_yOuaRupWcvvBB5tNnjVrtDllXqg6x4Q", output="/content/narration.zip", quiet=False)
    !unzip -q /content/narration.zip -d /content/narration
    !rm /content/narration.zip
    print(f"Loaded {len(_os.listdir('/content/narration'))} narration segments")
else:
    print("Narration audio already loaded.")

from IPython.display import Audio, display
display(Audio("/content/narration/03_00_intro.mp3"))


# Building a VAE from Scratch in PyTorch

*Part 3 of 4 in the Vizuara series on Variational Autoencoders From Scratch*

*Estimated time: 50 minutes*

In Notebook 1, we explored why regular autoencoders fail as generative models. In Notebook 2, we built the mathematical foundations — the ELBO, KL divergence, and the reparameterization trick. Now we put all of that theory into working code.

By the end of this notebook, you will have:
- A fully functional VAE that generates new handwritten digits from pure noise
- Clear understanding of every design choice in the architecture
- Training curves showing the delicate balance between reconstruction and regularization
- Side-by-side comparisons of original digits and their reconstructions

Let us build it.

In [ ]:
#@title 🎧 Listen: Why Matter
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_02_why_matter.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## Section 1: Why Does This Matter?

Here is what we are about to accomplish: we will write approximately 60 lines of PyTorch that, after 20 epochs of training on MNIST, can do something remarkable. We will sample a random point from a standard normal distribution — pure noise — feed it through a neural network, and out comes a handwritten digit that looks like a human wrote it.

This is fundamentally different from a regular autoencoder. A regular autoencoder compresses and reconstructs. A VAE *generates*. It learns the underlying distribution of handwritten digits so well that it can create new ones that have never existed before.

The key ingredient that makes this possible? Instead of encoding each image to a single point in latent space, we encode it to a *distribution*. And instead of a simple reconstruction loss, we add a regularization term (KL divergence) that shapes the latent space into something we can sample from.

Let us build each piece, one component at a time.

In [ ]:
#@title 🎧 Listen: Vae Diff Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_03_vae_diff_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 Listen: Diff1 Dist
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_04_diff1_dist.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 Listen: Diff2 Reparam
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_05_diff2_reparam.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 Listen: Diff3 Elbo
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_06_diff3_elbo.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## Section 2: Building Intuition — What Makes a VAE Different?

Before we write a single line of code, let us recap the three critical differences between a regular autoencoder and a VAE. Each one directly affects how we build the architecture.

### Difference 1: Distributions, Not Points

A regular autoencoder maps input $x$ to a single latent vector $z$. The encoder is deterministic — the same input always produces the same code.

A VAE encoder outputs two vectors: a mean $\mu$ and a log-variance $\log(\sigma^2)$. Together, these define a Gaussian distribution in latent space. The same input maps to a *cloud* of possible latent codes.

**Architecture implication:** The encoder needs two output heads instead of one. We will build `fc_mu` and `fc_logvar` as separate linear layers.

### Difference 2: The Reparameterization Trick

We cannot backpropagate through a random sampling operation. If we just did `z = torch.randn(...)`, gradients would not flow from the decoder back to the encoder. The reparameterization trick separates the randomness from the learnable parameters:

$$z = \mu + \sigma \cdot \epsilon, \quad \epsilon \sim \mathcal{N}(0, I)$$

The randomness is in $\epsilon$, which has no learnable parameters. The encoder outputs ($\mu$ and $\sigma$) are deterministic functions of the input, so gradients flow through them normally.

**Architecture implication:** We need a `reparameterize` method that takes $\mu$ and $\log(\sigma^2)$ and returns a sampled $z$.

### Difference 3: The ELBO Loss

A regular autoencoder minimizes reconstruction error alone. A VAE minimizes the **Evidence Lower Bound (ELBO)**, which has two terms:

$$\mathcal{L} = \underbrace{\text{BCE}(x, \hat{x})}_{\text{reconstruction}} + \underbrace{D_{KL}(q(z|x) \| p(z))}_{\text{regularization}}$$

The reconstruction term asks: "How well did we recreate the input?" The KL term asks: "How close is the encoder distribution to a standard normal?" These two forces pull in opposite directions, and their balance determines the quality of the generative model.

**Architecture implication:** Our loss function must compute and return both terms separately so we can monitor the balance during training.

In [ ]:
#@title 🎧 Listen: Math Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_07_math_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 Listen: Math Logvar
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_08_math_logvar.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 Listen: Math Reparam
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_09_math_reparam.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 Listen: Math Loss
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_10_math_loss.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## Section 3: The Mathematics for Implementation

Let us write down the three equations we will implement, focusing on the choices that matter for code.

### 3.1 The Encoder Distribution

The encoder defines a Gaussian posterior over the latent space:

$$q(z|x) = \mathcal{N}(\mu(x), \text{diag}(\sigma^2(x)))$$

where $\mu(x)$ and $\sigma^2(x)$ are neural network outputs.

**Why log-variance instead of $\sigma$ directly?**

This is a crucial design choice. The standard deviation $\sigma$ must be positive. If we output $\sigma$ directly, we would need to constrain the network output (e.g., with a softplus). Instead, we output $\log(\sigma^2)$, which can be any real number — positive or negative. To recover $\sigma$, we compute:

$$\sigma = \exp\left(\frac{1}{2} \log(\sigma^2)\right)$$

This is numerically stable because the exponential naturally ensures positivity, and operating in log-space avoids very large or very small variance values that could destabilize training.

### 3.2 The Reparameterization

$$z = \mu + \exp\left(\frac{1}{2} \log(\sigma^2)\right) \cdot \epsilon, \quad \epsilon \sim \mathcal{N}(0, I)$$

In code, this is just three lines. But those three lines are what make a VAE trainable.

### 3.3 The Loss Function

**Reconstruction loss** — We use Binary Cross-Entropy (BCE) because MNIST pixels are values between 0 and 1. Our decoder outputs pixel probabilities through a sigmoid, and BCE measures how well these probabilities match the actual pixel values:

$$\mathcal{L}_{\text{recon}} = -\sum_{i=1}^{784} \left[ x_i \log(\hat{x}_i) + (1 - x_i) \log(1 - \hat{x}_i) \right]$$

**KL divergence** — For two Gaussians (our posterior vs. the standard normal prior), the KL divergence has a closed-form solution:

$$\mathcal{L}_{\text{KL}} = -\frac{1}{2} \sum_{j=1}^{d} \left( 1 + \log(\sigma_j^2) - \mu_j^2 - \sigma_j^2 \right)$$

**Why `reduction='sum'`?** We sum over all 784 pixels and all latent dimensions rather than averaging. This keeps the reconstruction and KL terms at comparable scales. If we averaged the reconstruction term (dividing by 784) but summed the KL term (over just 2 dimensions), the KL term would dominate and the model would ignore reconstruction entirely.

In [ ]:
#@title 🎧 Transition: Build Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_11_build_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## Section 4: Let Us Build It — Component by Component

### 4.1 Setup and Data Loading

In [ ]:
#@title 🎧 Code Walkthrough: Imports Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_12_imports_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

# Reproducibility
torch.manual_seed(42)
np.random.seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
#@title 🎧 Code Walkthrough: Data Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_13_data_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Load MNIST
# We normalize to [0, 1] — important for BCE loss!
transform = transforms.Compose([
    transforms.ToTensor(),  # Converts to [0, 1] range
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2, pin_memory=True)

print(f"Training samples: {len(train_dataset):,}")
print(f"Test samples:     {len(test_dataset):,}")
print(f"Image shape:      {train_dataset[0][0].shape}")
print(f"Pixel range:      [{train_dataset[0][0].min():.1f}, {train_dataset[0][0].max():.1f}]")

In [ ]:
#@title 🎧 Listen: Visualize Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_14_visualize_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Let us visualize some training images so we know what our VAE will be learning to reconstruct and generate.

In [ ]:
#@title 🎧 What to Look For: Visualize Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_15_visualize_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Visualize sample training images
fig, axes = plt.subplots(2, 8, figsize=(14, 4))
fig.suptitle('MNIST Training Samples — What Our VAE Will Learn', fontsize=14, fontweight='bold')
for i, ax in enumerate(axes.flat):
    img, label = train_dataset[i]
    ax.imshow(img.squeeze(), cmap='gray')
    ax.set_title(str(label), fontsize=10)
    ax.axis('off')
plt.tight_layout()
plt.show()

print(f"\nEach image is 28x28 = 784 pixels")
print(f"Our VAE will compress these 784 values into just 2 latent dimensions")
print(f"That is a compression ratio of {784/2:.0f}:1!")

In [ ]:
#@title 🎧 Listen: Encoder Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_16_encoder_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.2 The Encoder

The encoder takes a flattened 784-dimensional image and maps it to two vectors: the mean $\mu$ and the log-variance $\log(\sigma^2)$ of the latent distribution. Notice the **two-headed output** — this is what makes it a VAE encoder rather than a regular autoencoder encoder.

Why two separate linear layers at the end? Because $\mu$ and $\log(\sigma^2)$ represent fundamentally different quantities. The mean tells us *where* in latent space this image lives. The log-variance tells us *how uncertain* the encoder is about that location. These should be independent outputs that the network learns separately.

In [ ]:
#@title 🎧 Code Walkthrough: Encoder Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_17_encoder_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
class Encoder(nn.Module):
    """VAE Encoder: maps input x to latent distribution parameters (mu, logvar).

    Architecture:
        784 -> 512 -> 256 -> (mu: latent_dim, logvar: latent_dim)

    The two-headed output is the key difference from a regular autoencoder.
    """
    def __init__(self, input_dim=784, hidden_dim=512, latent_dim=2):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 256)
        self.fc_mu = nn.Linear(256, latent_dim)       # Mean of q(z|x)
        self.fc_logvar = nn.Linear(256, latent_dim)   # Log-variance of q(z|x)

    def forward(self, x):
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar

In [ ]:
#@title 🎧 Listen: Test Encoder Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_18_test_encoder_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Let us verify the encoder and inspect its outputs.

In [ ]:
#@title 🎧 Code Walkthrough: Test Encoder Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_19_test_encoder_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Test the encoder
encoder = Encoder(input_dim=784, hidden_dim=512, latent_dim=2).to(device)
dummy_input = torch.randn(4, 784).to(device)  # Batch of 4 flattened images

mu, logvar = encoder(dummy_input)
print("Encoder Test")
print("=" * 40)
print(f"Input shape:   {dummy_input.shape}")    # (4, 784)
print(f"Mu shape:      {mu.shape}")              # (4, 2)
print(f"Logvar shape:  {logvar.shape}")          # (4, 2)
print(f"\nSample mu:     {mu[0].detach().cpu().numpy()}")
print(f"Sample logvar: {logvar[0].detach().cpu().numpy()}")

# Parameter count
enc_params = sum(p.numel() for p in encoder.parameters())
print(f"\nEncoder parameters: {enc_params:,}")

In [ ]:
#@title 🎧 Listen: Logvar Interpret
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_20_logvar_interpret.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Notice that `logvar` values can be positive or negative — they are not constrained. A logvar of -2 means $\sigma^2 = e^{-2} \approx 0.14$ (the encoder is fairly certain), while a logvar of 2 means $\sigma^2 = e^{2} \approx 7.4$ (high uncertainty). The network learns these values through backpropagation.

In [ ]:
#@title 🎧 Listen: Decoder Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_21_decoder_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.3 The Decoder

The decoder takes a latent vector $z$ and maps it back to a 784-dimensional output — a reconstructed image. The critical detail is the **sigmoid activation** at the output.

Why sigmoid? Because MNIST pixels are in the range $[0, 1]$, and we use Binary Cross-Entropy (BCE) as our reconstruction loss. BCE expects both the target and the prediction to be probabilities in $[0, 1]$. The sigmoid squashes the raw network output into exactly this range. You can think of each output value as the *probability* that the corresponding pixel is "on."

In [ ]:
#@title 🎧 Code Walkthrough: Decoder Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_22_decoder_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
class Decoder(nn.Module):
    """VAE Decoder: maps latent vector z back to image space.

    Architecture:
        latent_dim -> 256 -> 512 -> 784 (with sigmoid)

    The sigmoid output gives pixel probabilities in [0, 1],
    which is required for Binary Cross-Entropy loss.
    """
    def __init__(self, latent_dim=2, hidden_dim=512, output_dim=784):
        super().__init__()
        self.fc1 = nn.Linear(latent_dim, 256)
        self.fc2 = nn.Linear(256, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)

    def forward(self, z):
        h = F.relu(self.fc1(z))
        h = F.relu(self.fc2(h))
        return torch.sigmoid(self.fc3(h))

In [ ]:
#@title 🎧 Code Walkthrough: Test Decoder Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_23_test_decoder_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Test the decoder
decoder = Decoder(latent_dim=2, hidden_dim=512, output_dim=784).to(device)
dummy_z = torch.randn(4, 2).to(device)  # Batch of 4 latent vectors

x_recon = decoder(dummy_z)
print("Decoder Test")
print("=" * 40)
print(f"Input shape (z): {dummy_z.shape}")      # (4, 2)
print(f"Output shape:    {x_recon.shape}")       # (4, 784)
print(f"Output range:    [{x_recon.min().item():.4f}, {x_recon.max().item():.4f}]")
print(f"  (Should be in [0, 1] thanks to sigmoid)")

# Parameter count
dec_params = sum(p.numel() for p in decoder.parameters())
print(f"\nDecoder parameters: {dec_params:,}")

In [ ]:
#@title 🎧 Listen: Vae Class Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_24_vae_class_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.4 The VAE Class — Connecting Everything with Reparameterization

Now we assemble the encoder and decoder into the full VAE. The `reparameterize` method is the bridge between them — it takes the encoder's distribution parameters and produces a concrete latent vector that we can feed to the decoder, while keeping the computation graph intact for backpropagation.

In [ ]:
#@title 🎧 Code Walkthrough: Vae Class Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_25_vae_class_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
class VAE(nn.Module):
    """Complete Variational Autoencoder.

    Pipeline:
        x -> Encoder -> (mu, logvar) -> Reparameterize -> z -> Decoder -> x_recon

    The reparameterization trick is what makes this trainable:
        z = mu + exp(0.5 * logvar) * epsilon,  epsilon ~ N(0, I)
    """
    def __init__(self, input_dim=784, hidden_dim=512, latent_dim=2):
        super().__init__()
        self.encoder = Encoder(input_dim, hidden_dim, latent_dim)
        self.decoder = Decoder(latent_dim, hidden_dim, input_dim)

    def reparameterize(self, mu, logvar):
        """Sample z from q(z|x) using the reparameterization trick.

        Instead of sampling z ~ N(mu, sigma^2), we compute:
            z = mu + sigma * epsilon,  where epsilon ~ N(0, I)

        This makes z a deterministic function of mu, logvar, and epsilon,
        so gradients can flow through mu and logvar.
        """
        std = torch.exp(0.5 * logvar)   # sigma = exp(0.5 * log(sigma^2))
        eps = torch.randn_like(std)      # epsilon ~ N(0, I)
        return mu + std * eps

    def forward(self, x):
        mu, logvar = self.encoder(x)
        z = self.reparameterize(mu, logvar)
        x_recon = self.decoder(z)
        return x_recon, mu, logvar

In [ ]:
#@title 🎧 Code Walkthrough: Test Vae Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_26_test_vae_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Test the full VAE pipeline
vae_test = VAE(input_dim=784, hidden_dim=512, latent_dim=2).to(device)
dummy_x = torch.randn(4, 784).clamp(0, 1).to(device)

x_recon, mu, logvar = vae_test(dummy_x)

print("Full VAE Pipeline Test")
print("=" * 40)
print(f"Input shape:   {dummy_x.shape}")       # (4, 784)
print(f"Recon shape:   {x_recon.shape}")        # (4, 784)
print(f"Mu shape:      {mu.shape}")              # (4, 2)
print(f"Logvar shape:  {logvar.shape}")          # (4, 2)
print(f"Recon range:   [{x_recon.min().item():.4f}, {x_recon.max().item():.4f}]")

total_params = sum(p.numel() for p in vae_test.parameters())
print(f"\nTotal VAE parameters: {total_params:,}")

In [ ]:
#@title 🎧 Listen: Arch Summary Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_27_arch_summary_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Let us visualize the architecture to make sure we understand the full data flow.

In [ ]:
#@title 🎧 Code Walkthrough: Arch Summary Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_28_arch_summary_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Architecture summary
print("=" * 60)
print("              VAE ARCHITECTURE SUMMARY")
print("=" * 60)
print()
print("  ENCODER")
print("  -------")
print("  Input (784) ──> Linear(784, 512) ──> ReLU")
print("                  Linear(512, 256) ──> ReLU")
print("                       ├──> Linear(256, 2) ──> mu")
print("                       └──> Linear(256, 2) ──> logvar")
print()
print("  REPARAMETERIZE")
print("  --------------")
print("  z = mu + exp(0.5 * logvar) * epsilon")
print("  epsilon ~ N(0, I)")
print()
print("  DECODER")
print("  -------")
print("  z (2) ──> Linear(2, 256) ──> ReLU")
print("            Linear(256, 512) ──> ReLU")
print("            Linear(512, 784) ──> Sigmoid ──> x_recon")
print()
print("=" * 60)

# Detailed parameter breakdown
enc_params = sum(p.numel() for p in vae_test.encoder.parameters())
dec_params = sum(p.numel() for p in vae_test.decoder.parameters())

print(f"  Encoder parameters:  {enc_params:>10,}")
print(f"  Decoder parameters:  {dec_params:>10,}")
print(f"  Total parameters:    {total_params:>10,}")
print(f"  Model size:          {total_params * 4 / 1024 / 1024:.2f} MB (float32)")
print("=" * 60)

In [ ]:
#@title 🎧 Listen: Loss Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_29_loss_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.5 The Loss Function

The VAE loss has two terms that work in tension:

1. **Reconstruction loss (BCE):** Pushes the model to faithfully reproduce the input. This term wants the encoder to memorize each image precisely — use the full capacity of the latent space, spread points apart, make each code unique.

2. **KL divergence:** Pushes the encoder distribution toward a standard normal $\mathcal{N}(0, I)$. This term wants all codes to cluster near the origin with unit variance — the opposite of what reconstruction wants.

The balance between these forces is what gives a VAE its generative power. The reconstruction term ensures the latent space is *informative* (different digits map to different regions). The KL term ensures it is *regular* (no gaps, smooth transitions, easy to sample from).

In [ ]:
#@title 🎧 Code Walkthrough: Loss Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_30_loss_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
def vae_loss(x_recon, x, mu, logvar):
    """Compute the VAE loss = Reconstruction + KL Divergence.

    Args:
        x_recon: Reconstructed images from decoder (B, 784), values in [0, 1]
        x: Original images (B, 784), values in [0, 1]
        mu: Encoder mean (B, latent_dim)
        logvar: Encoder log-variance (B, latent_dim)

    Returns:
        total_loss: Scalar, the sum of reconstruction and KL losses
        recon_loss: Scalar, BCE reconstruction term
        kl_loss: Scalar, KL divergence term

    Note: We use reduction='sum' to keep recon and KL at comparable scales.
    """
    # Reconstruction: how well did we recreate the input?
    recon_loss = F.binary_cross_entropy(x_recon, x, reduction='sum')

    # KL divergence: how far is q(z|x) from N(0, I)?
    # Closed-form for Gaussian: -0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return recon_loss + kl_loss, recon_loss, kl_loss

In [ ]:
#@title 🎧 Code Walkthrough: Test Loss Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_31_test_loss_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Test the loss function
x_test = torch.rand(4, 784).to(device)  # Fake input in [0, 1]
x_recon_test, mu_test, logvar_test = vae_test(x_test)

total, recon, kl = vae_loss(x_recon_test, x_test, mu_test, logvar_test)
print("Loss Function Test")
print("=" * 40)
print(f"Reconstruction loss: {recon.item():>10.2f}")
print(f"KL divergence:       {kl.item():>10.2f}")
print(f"Total loss:          {total.item():>10.2f}")
print(f"\nRecon / Total:       {recon.item()/total.item():>10.1%}")
print(f"KL / Total:          {kl.item()/total.item():>10.1%}")
print("\n(At initialization, KL should be small — the encoder")
print(" hasn't learned to use the latent space yet)")

In [ ]:
#@title 🎧 Before You Start: Todo1 Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_32_todo1_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## Section 5: Your Turn!

Before we run the full training, let us make sure you deeply understand two of the most important pieces. Implement them yourself.

### TODO 1: Implement the Reparameterization Trick

Given $\mu$ and $\log(\sigma^2)$ from the encoder, sample a latent vector $z$ using the reparameterization trick.

**Recall the formula:**

$$z = \mu + \sigma \cdot \epsilon, \quad \text{where } \sigma = \exp\left(\frac{1}{2} \log(\sigma^2)\right), \quad \epsilon \sim \mathcal{N}(0, I)$$

In [ ]:
#@title 🎧 Before You Start: Todo1 Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_33_todo1_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
def reparameterize_todo(mu, logvar):
    """
    Sample z from q(z|x) using the reparameterization trick.

    Steps:
      1. Compute std from logvar:  std = exp(0.5 * logvar)
      2. Sample epsilon:           eps = torch.randn_like(std)
      3. Compute z:                z = mu + std * eps

    Args:
        mu: (B, latent_dim) — mean of the encoder distribution
        logvar: (B, latent_dim) — log-variance of the encoder distribution

    Returns:
        z: (B, latent_dim) — sampled latent vector

    Hints:
        - torch.exp() computes element-wise exponential
        - torch.randn_like(tensor) samples from N(0,1) with same shape as tensor
        - All operations are element-wise
    """
    # ============ YOUR CODE HERE ============
    # TODO: Implement the 3 steps above

    raise NotImplementedError("Implement the reparameterization trick!")
    # =========================================

In [ ]:
#@title 🎧 Before You Start: Todo1 After
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_34_todo1_after.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# --- Verification for TODO 1 ---
# Uncomment after implementing:

# torch.manual_seed(42)
# test_mu = torch.tensor([[1.0, -1.0], [0.5, 0.5]])
# test_logvar = torch.tensor([[0.0, 0.0], [1.0, -1.0]])  # sigma^2 = 1, 1, e, 1/e
#
# z = reparameterize_todo(test_mu, test_logvar)
# print(f"Mu:     {test_mu}")
# print(f"Logvar: {test_logvar}")
# print(f"z:      {z}")
# print(f"z shape: {z.shape}")
# assert z.shape == (2, 2), f"Expected shape (2, 2), got {z.shape}"
#
# # Check that z is close to mu (on average) — run many samples
# torch.manual_seed(0)
# zs = torch.stack([reparameterize_todo(test_mu, test_logvar) for _ in range(10000)])
# mean_z = zs.mean(dim=0)
# print(f"\nMean of 10,000 samples: {mean_z}")
# print(f"Should be close to mu:  {test_mu}")
# assert torch.allclose(mean_z, test_mu, atol=0.1), "Mean of samples should be close to mu!"
# print("\nTODO 1: PASSED!")

In [ ]:
#@title 🎧 Before You Start: Todo2 Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_35_todo2_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### TODO 2: Implement the VAE Loss Function

Given the reconstruction, original input, $\mu$, and $\log(\sigma^2)$, compute the two loss terms.

**Recall the formulas:**

Reconstruction: $\mathcal{L}_{\text{recon}} = \text{BCE}(\hat{x}, x, \text{reduction}=\text{sum})$

KL divergence: $\mathcal{L}_{\text{KL}} = -\frac{1}{2} \sum (1 + \log(\sigma^2) - \mu^2 - \sigma^2)$

In [ ]:
#@title 🎧 Before You Start: Todo2 Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_36_todo2_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
def vae_loss_todo(x_recon, x, mu, logvar):
    """
    Compute the VAE loss: Reconstruction + KL Divergence.

    Steps:
      1. Compute recon_loss using F.binary_cross_entropy with reduction='sum'
      2. Compute kl_loss: -0.5 * sum(1 + logvar - mu^2 - exp(logvar))
      3. Return total_loss, recon_loss, kl_loss

    Args:
        x_recon: (B, 784) — decoder output, values in [0, 1]
        x: (B, 784) — original input, values in [0, 1]
        mu: (B, latent_dim) — encoder mean
        logvar: (B, latent_dim) — encoder log-variance

    Returns:
        total_loss: recon_loss + kl_loss
        recon_loss: BCE reconstruction term
        kl_loss: KL divergence term

    Hints:
        - F.binary_cross_entropy(pred, target, reduction='sum')
        - mu.pow(2) computes element-wise square
        - logvar.exp() computes element-wise exponential
        - torch.sum() sums all elements
    """
    # ============ YOUR CODE HERE ============
    # TODO: Implement the 3 steps above

    raise NotImplementedError("Implement the VAE loss function!")
    # =========================================

In [ ]:
#@title 🎧 Before You Start: Todo2 After
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_37_todo2_after.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# --- Verification for TODO 2 ---
# Uncomment after implementing:

# # Create known inputs
# test_x = torch.tensor([[0.8, 0.2, 0.5], [0.1, 0.9, 0.3]])
# test_x_recon = torch.tensor([[0.7, 0.3, 0.6], [0.2, 0.8, 0.4]])
# test_mu = torch.tensor([[0.5, -0.5], [1.0, 0.0]])
# test_logvar = torch.tensor([[0.0, 0.0], [0.5, -0.5]])
#
# total, recon, kl = vae_loss_todo(test_x_recon, test_x, test_mu, test_logvar)
#
# # Check against reference implementation
# ref_recon = F.binary_cross_entropy(test_x_recon, test_x, reduction='sum')
# ref_kl = -0.5 * torch.sum(1 + test_logvar - test_mu.pow(2) - test_logvar.exp())
#
# print(f"Your recon loss:     {recon.item():.4f}")
# print(f"Reference recon:     {ref_recon.item():.4f}")
# print(f"Your KL loss:        {kl.item():.4f}")
# print(f"Reference KL:        {ref_kl.item():.4f}")
# print(f"Your total:          {total.item():.4f}")
# print(f"Reference total:     {(ref_recon + ref_kl).item():.4f}")
#
# assert torch.allclose(recon, ref_recon, atol=1e-4), "Reconstruction loss mismatch!"
# assert torch.allclose(kl, ref_kl, atol=1e-4), "KL divergence mismatch!"
# print("\nTODO 2: PASSED!")

In [ ]:
#@title 🎧 Transition: Training Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_38_training_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## Section 6: Training

Now let us train our VAE. We will track three losses separately — total, reconstruction, and KL — so we can observe the balance between them. This balance is one of the most important things to monitor when training a VAE.

In [ ]:
#@title 🎧 Code Walkthrough: Training Setup Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_39_training_setup_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Create the model
torch.manual_seed(42)
model = VAE(input_dim=784, hidden_dim=512, latent_dim=2).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

total_params = sum(p.numel() for p in model.parameters())
print("=" * 55)
print("           VAE TRAINING CONFIGURATION")
print("=" * 55)
print(f"  Model:         VAE (latent_dim=2)")
print(f"  Parameters:    {total_params:,}")
print(f"  Optimizer:     Adam (lr=1e-3)")
print(f"  Batch size:    128")
print(f"  Epochs:        20")
print(f"  Device:        {device}")
print(f"  Dataset:       MNIST (60,000 training images)")
print("=" * 55)

In [ ]:
#@title 🎧 Listen: Recon Snapshots Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_40_recon_snapshots_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


We will also save sample reconstructions at regular intervals during training, so we can visualize how the model improves over time.

In [ ]:
#@title 🎧 Code Walkthrough: Training Loop Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_41_training_loop_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Get a fixed batch of test images for tracking reconstruction quality
fixed_test_images, fixed_test_labels = next(iter(test_loader))
fixed_test_images = fixed_test_images[:16].to(device)  # 16 images for visualization
fixed_test_flat = fixed_test_images.view(-1, 784)

# Training loop
EPOCHS = 20
history = {
    'total_loss': [], 'recon_loss': [], 'kl_loss': []
}
reconstruction_snapshots = {}  # Save reconstructions at certain epochs

print(f"{'Epoch':>5} | {'Total Loss':>10} | {'Recon Loss':>10} | {'KL Loss':>10} | {'Recon %':>7}")
print("-" * 60)

for epoch in range(1, EPOCHS + 1):
    model.train()
    epoch_total = 0
    epoch_recon = 0
    epoch_kl = 0
    n_batches = 0

    for batch_images, _ in train_loader:
        # Flatten images: (B, 1, 28, 28) -> (B, 784)
        batch_images = batch_images.view(-1, 784).to(device)

        # Forward pass
        x_recon, mu, logvar = model(batch_images)

        # Compute loss
        total, recon, kl = vae_loss(x_recon, batch_images, mu, logvar)

        # Backward pass
        optimizer.zero_grad()
        total.backward()
        optimizer.step()

        # Track losses (normalize by batch size for cleaner numbers)
        epoch_total += total.item() / len(batch_images)
        epoch_recon += recon.item() / len(batch_images)
        epoch_kl += kl.item() / len(batch_images)
        n_batches += 1

    # Average over batches
    avg_total = epoch_total / n_batches
    avg_recon = epoch_recon / n_batches
    avg_kl = epoch_kl / n_batches

    history['total_loss'].append(avg_total)
    history['recon_loss'].append(avg_recon)
    history['kl_loss'].append(avg_kl)

    recon_pct = avg_recon / avg_total * 100

    print(f"{epoch:>5d} | {avg_total:>10.2f} | {avg_recon:>10.2f} | {avg_kl:>10.2f} | {recon_pct:>6.1f}%")

    # Save reconstruction snapshots at key epochs
    if epoch in [1, 5, 10, 15, 20]:
        model.eval()
        with torch.no_grad():
            recon_snap, _, _ = model(fixed_test_flat)
            reconstruction_snapshots[epoch] = recon_snap.cpu().view(-1, 1, 28, 28)

print(f"\nTraining complete!")
print(f"Final total loss: {history['total_loss'][-1]:.2f}")

In [ ]:
#@title 🎧 Listen: Visualize Recon Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_42_visualize_recon_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Let us visualize how reconstructions improve across training. This gives us a concrete sense of what the model is learning at each stage.

In [ ]:
#@title 🎧 What to Look For: Visualize Recon Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_43_visualize_recon_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Visualize reconstruction quality across training
snapshot_epochs = sorted(reconstruction_snapshots.keys())
n_show = 8  # Number of images to show

fig, axes = plt.subplots(len(snapshot_epochs) + 1, n_show, figsize=(14, 3 * (len(snapshot_epochs) + 1)))
fig.suptitle('Reconstruction Quality Across Training', fontsize=16, fontweight='bold', y=1.02)

# Row 0: Original images
for j in range(n_show):
    axes[0, j].imshow(fixed_test_images[j].cpu().squeeze(), cmap='gray')
    axes[0, j].axis('off')
    if j == 0:
        axes[0, j].set_ylabel('Original', fontsize=12, fontweight='bold', rotation=0, labelpad=60, va='center')

# Remaining rows: Reconstructions at each snapshot
for i, ep in enumerate(snapshot_epochs):
    for j in range(n_show):
        axes[i+1, j].imshow(reconstruction_snapshots[ep][j].squeeze(), cmap='gray')
        axes[i+1, j].axis('off')
        if j == 0:
            axes[i+1, j].set_ylabel(f'Epoch {ep}', fontsize=12, fontweight='bold', rotation=0, labelpad=60, va='center')

plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 Listen: Recon Progression
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_44_recon_progression.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Notice the progression:
- **Epoch 1:** The reconstructions are blurry blobs — the model has barely learned anything.
- **Epoch 5:** Rough digit shapes emerge. The model is learning the most important structures.
- **Epoch 10-15:** Digits become recognizable. Details sharpen as the model refines its latent representations.
- **Epoch 20:** Reconstructions are reasonably clear, though still softer than the originals. This "blurriness" is a well-known characteristic of VAEs — a consequence of the probabilistic decoding.

In [ ]:
#@title 🎧 Transition: Results Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_45_results_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## Section 7: Results

### 7.1 Training Curves

Let us plot the three loss components together. The relationship between reconstruction loss and KL divergence tells us a lot about what the model has learned.

In [ ]:
#@title 🎧 What to Look For: Training Curves Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_46_training_curves_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Training curves: total, reconstruction, and KL loss
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

epochs_range = range(1, EPOCHS + 1)

# Plot 1: Total loss
axes[0].plot(epochs_range, history['total_loss'], color='#1565C0', linewidth=2.5, marker='o', markersize=4)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss (per sample)', fontsize=12)
axes[0].set_title('Total Loss', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Plot 2: Reconstruction vs KL (stacked or overlaid)
axes[1].plot(epochs_range, history['recon_loss'], color='#E65100', linewidth=2.5, marker='s', markersize=4, label='Reconstruction (BCE)')
axes[1].plot(epochs_range, history['kl_loss'], color='#2E7D32', linewidth=2.5, marker='^', markersize=4, label='KL Divergence')
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Loss (per sample)', fontsize=12)
axes[1].set_title('Loss Components', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

# Plot 3: KL as percentage of total
kl_pct = [kl / total * 100 for kl, total in zip(history['kl_loss'], history['total_loss'])]
axes[2].plot(epochs_range, kl_pct, color='#6A1B9A', linewidth=2.5, marker='D', markersize=4)
axes[2].set_xlabel('Epoch', fontsize=12)
axes[2].set_ylabel('KL / Total (%)', fontsize=12)
axes[2].set_title('KL Divergence Share', fontsize=14, fontweight='bold')
axes[2].grid(True, alpha=0.3)
axes[2].axhline(y=kl_pct[-1], color='gray', linestyle='--', alpha=0.5)
axes[2].annotate(f'{kl_pct[-1]:.1f}%', xy=(EPOCHS, kl_pct[-1]),
                 fontsize=10, fontweight='bold', color='#6A1B9A',
                 xytext=(EPOCHS - 3, kl_pct[-1] + 2))

plt.suptitle('VAE Training Dynamics', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 Listen: Curves Analysis
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_47_curves_analysis.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


A few things to notice about these curves:

- **The KL divergence increases during early training.** At initialization, the encoder outputs are near zero (close to the prior), so KL is low. As the model learns to use the latent space meaningfully, the encoder distributions move away from the prior, and KL rises. This is healthy — it means the model is learning to encode information.

- **Reconstruction loss drops rapidly then plateaus.** The biggest visual improvements happen in the first few epochs. Later epochs mainly refine details.

- **The KL share stabilizes.** In a healthy VAE, the KL term settles at a relatively small fraction of the total loss (typically 5-20%). If KL were zero, it would mean the encoder ignores the input entirely (posterior collapse). If KL dominated, it would mean the model cannot reconstruct anything.

In [ ]:
#@title 🎧 Listen: Recon Quality Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_48_recon_quality_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 7.2 Reconstruction Quality

Let us do a thorough comparison: 16 test images side-by-side with their reconstructions.

In [ ]:
#@title 🎧 What to Look For: Recon Quality Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_49_recon_quality_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Reconstruction comparison on test set
model.eval()
test_images, test_labels = next(iter(test_loader))
test_flat = test_images[:16].view(-1, 784).to(device)

with torch.no_grad():
    recons, _, _ = model(test_flat)
    recons = recons.view(-1, 1, 28, 28).cpu()

fig, axes = plt.subplots(4, 8, figsize=(16, 8))
fig.suptitle('Original (Top) vs Reconstruction (Bottom) — 16 Test Digits',
             fontsize=16, fontweight='bold', y=1.02)

for i in range(16):
    row = (i // 8) * 2
    col = i % 8

    # Original
    axes[row, col].imshow(test_images[i].squeeze(), cmap='gray')
    axes[row, col].axis('off')
    axes[row, col].set_title(f'{test_labels[i].item()}', fontsize=10, color='#1565C0')

    # Reconstruction
    axes[row+1, col].imshow(recons[i].squeeze(), cmap='gray')
    axes[row+1, col].axis('off')

# Add row labels
axes[0, 0].set_ylabel('Original', fontsize=12, fontweight='bold', rotation=0, labelpad=55, va='center')
axes[1, 0].set_ylabel('VAE', fontsize=12, fontweight='bold', rotation=0, labelpad=55, va='center')
axes[2, 0].set_ylabel('Original', fontsize=12, fontweight='bold', rotation=0, labelpad=55, va='center')
axes[3, 0].set_ylabel('VAE', fontsize=12, fontweight='bold', rotation=0, labelpad=55, va='center')

plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 Listen: Blurriness Explain
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_50_blurriness_explain.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


The reconstructions capture the overall structure of each digit — the right number, roughly the right style, the correct position. But they are noticeably softer and blurrier than the originals. This is not a bug; it is a fundamental property of VAEs.

**Why the blurriness?** The decoder must produce an output that is a good reconstruction for *any* latent vector in the neighborhood of the encoded point (because of the noise injected by reparameterization). It hedges its bets by producing the average of plausible outputs, which looks blurry. This is the price we pay for having a smooth, sample-able latent space.

We will explore techniques to address this blurriness in Notebook 4.

In [ ]:
#@title 🎧 Transition: Final Output Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_51_final_output_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## Section 8: Final Output

Let us create the definitive figure for this notebook: a beautiful grid comparing originals and reconstructions, plus a summary of what we built.

In [ ]:
#@title 🎧 What to Look For: Final Recon Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_52_final_recon_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Final reconstruction comparison grid
model.eval()

# Get diverse digits: pick one of each 0-9
final_images = []
final_labels = []
for digit in range(10):
    for img, label in test_dataset:
        if label == digit:
            final_images.append(img)
            final_labels.append(label)
            break

final_images = torch.stack(final_images).to(device)
final_flat = final_images.view(-1, 784)

with torch.no_grad():
    final_recons, _, _ = model(final_flat)
    final_recons = final_recons.view(-1, 1, 28, 28).cpu()

fig, axes = plt.subplots(2, 10, figsize=(16, 4))
fig.suptitle('VAE Reconstructions — One of Each Digit (0-9)', fontsize=16, fontweight='bold', y=1.05)

for i in range(10):
    axes[0, i].imshow(final_images[i].cpu().squeeze(), cmap='gray')
    axes[0, i].set_title(str(i), fontsize=12, fontweight='bold')
    axes[0, i].axis('off')

    axes[1, i].imshow(final_recons[i].squeeze(), cmap='gray')
    axes[1, i].axis('off')

axes[0, 0].set_ylabel('Original', fontsize=12, fontweight='bold', rotation=0, labelpad=55, va='center')
axes[1, 0].set_ylabel('VAE', fontsize=12, fontweight='bold', rotation=0, labelpad=55, va='center')

plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 Code Walkthrough: Save Model
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_53_save_model.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Save the trained model for Notebook 4
torch.save(model.state_dict(), 'vae_mnist.pth')
print(f"Model saved to 'vae_mnist.pth'")
print(f"File size: {sum(p.numel() for p in model.parameters()) * 4 / 1024:.1f} KB")

In [ ]:
#@title 🎧 Listen: Summary Print
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_54_summary_print.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
print("=" * 65)
print("  Congratulations! You have built a Variational Autoencoder")
print("  from scratch!")
print("=" * 65)
print()
print("  What we built:")
print("    - Encoder: 784 -> 512 -> 256 -> (mu, logvar) with 2 latent dims")
print("    - Decoder: 2 -> 256 -> 512 -> 784 with sigmoid output")
print("    - Reparameterization: z = mu + sigma * epsilon")
print("    - Loss: BCE reconstruction + KL divergence")
print()
print(f"  Training results:")
print(f"    - Final total loss:  {history['total_loss'][-1]:.2f}")
print(f"    - Final recon loss:  {history['recon_loss'][-1]:.2f}")
print(f"    - Final KL loss:     {history['kl_loss'][-1]:.2f}")
print()
print("  The model is saved to 'vae_mnist.pth' for Notebook 4.")
print()
print("=" * 65)

In [ ]:
#@title 🎧 Listen: Reflection Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_55_reflection_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## Section 9: Reflection and Next Steps

### Reflection Questions

Take a moment to think about what we have built.

**1. Why log-variance and not variance directly?**

We output $\log(\sigma^2)$ instead of $\sigma^2$ because the logarithm maps the positive-only variance to the full real line, which neural networks handle naturally. To recover $\sigma$ for the reparameterization trick, we compute $\exp(0.5 \cdot \log(\sigma^2))$. This exponential guarantees positivity without any additional constraints. It also provides better numerical stability — very small and very large variances are represented equally well in log-space.

**2. Why does the KL term prevent posterior collapse... but also risk it?**

The KL term penalizes the encoder for deviating from the prior $\mathcal{N}(0, I)$. If the KL weight is too strong (or the decoder is too powerful), the encoder might find it easier to just output $\mu = 0, \log(\sigma^2) = 0$ for every input, achieving zero KL. This is *posterior collapse* — the latent space carries no information, and the decoder must generate everything from scratch. Monitoring the KL loss during training helps catch this: if KL drops to near zero while reconstruction stays high, the model has collapsed.

**3. Why sum reduction instead of mean?**

With `reduction='sum'`, the reconstruction loss sums over all 784 pixels, producing a large number. The KL loss sums over 2 latent dimensions, producing a smaller number. If we used `reduction='mean'` for reconstruction (dividing by 784), the KL term would be relatively much larger and would dominate. The sum keeps the two terms at natural scales where the model balances them well.

In [ ]:
#@title 🎧 Listen: Next Steps
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_56_next_steps.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### What Comes Next — Notebook 4

We have trained a VAE, but we have only used it for reconstruction so far. The real power of a VAE is **generation** — creating new images that have never been seen before. In Notebook 4, we will:

1. **Explore the latent space** — Visualize where each digit class lives in the 2D latent space
2. **Generate new digits** — Sample from $\mathcal{N}(0, I)$ and decode to create novel images
3. **Interpolate between digits** — Walk smoothly through latent space and watch digits morph
4. **Build a latent space grid** — Systematically sample the 2D space to see what the decoder has learned at every location

The model we saved (`vae_mnist.pth`) will be loaded directly into Notebook 4, so everything we trained here carries forward.

In [ ]:
#@title 🎧 Listen: Closing Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_57_closing_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 Wrap-Up: Closing
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_58_closing.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
print("Next up: Notebook 4 — Exploring the Latent Space")
print()
print("We will load the model we just saved and use it to:")
print("  1. Visualize the latent space (where do 0s, 1s, 2s... live?)")
print("  2. Generate brand-new digits from random noise")
print("  3. Interpolate between digits (watch a 3 morph into an 8)")
print("  4. Build a complete grid of the latent space")
print()
print("See you in Notebook 4!")